<a href="https://colab.research.google.com/github/NiaRamadhani31/DIFG2130_B_25_2311531006/blob/main/Pratikum3/NaiveBayesModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Outlook: Cuaca saat kejadian (sunny = cerah, overcast = mendung, rainy = hujan).
Temperature: Suhu dalam derajat Fahrenheit.
Humidity: Kelembaban dalam persen.
Wind: Apakah ada angin (True = ada, False = tidak ada).
Play: Keputusan apakah bermain tenis atau tidak (Yes = bermain, No = tidak bermain).


In [14]:
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

dataset_dict = {
    'Outlook': ['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast', 'overcast', 'rainy', 'sunny', 'overcast', 'rainy', 'sunny', 'sunny', 'rainy', 'overcast', 'rainy', 'sunny', 'overcast', 'sunny', 'overcast', 'rainy', 'overcast'],
    'Temperature': [85.0, 80.0, 83.0, 70.0, 68.0, 65.0, 64.0, 72.0, 69.0, 75.0, 75.0, 72.0, 81.0, 71.0, 81.0, 74.0, 76.0, 78.0, 82.0, 67.0, 85.0, 73.0, 88.0, 77.0, 79.0, 80.0, 66.0, 84.0],
    'Humidity': [85.0, 90.0, 78.0, 96.0, 80.0, 70.0, 65.0, 95.0, 70.0, 80.0, 70.0, 90.0, 75.0, 80.0, 88.0, 92.0, 85.0, 75.0, 92.0, 90.0, 85.0, 88.0, 65.0, 70.0, 60.0, 95.0, 70.0, 78.0],
    'Wind': [False, True, False, False, False, True, True, False, False, False, True, True, False, True, True, False, False, True, False, True, True, False, True, False, False, True, False, False],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes']
}

df = pd.DataFrame(dataset_dict)

df.head()

,Outlook,Temperature,Humidity,Wind,Play
0,sunny,85.0,85.0,False,No
1,sunny,80.0,90.0,True,No
2,overcast,83.0,78.0,False,Yes
3,rainy,70.0,96.0,False,Yes
4,rainy,68.0,80.0,False,Yes


Kolom Outlook dikonversi ke variabel dummy

Kolom Outlook (kategori: sunny, overcast, rainy) diubah menjadi tiga kolom biner: overcast, rainy, dan sunny.
Nilainya adalah 1 jika baris tersebut sesuai dengan kategori, dan 0 jika tidak.
Kolom Wind dikonversi ke bilangan bulat (0 dan 1)

False menjadi 0, True menjadi 1.
Kolom Play dikonversi ke bilangan bulat (0 dan 1)

No menjadi 0, Yes menjadi 1.


In [16]:
df = pd.get_dummies(df, columns=['Outlook'], prefix='', prefix_sep='', dtype=int)

df['Wind'] = df['Wind'].astype(int)
df['Play'] = df['Play'].map({'Yes': 1, 'No': 0})

df.head()

,Temperature,Humidity,Wind,Play,overcast,rainy,sunny
0,85.0,85.0,0,0,0,0,1
1,80.0,90.0,1,0,0,0,1
2,83.0,78.0,0,1,1,0,0
3,70.0,96.0,0,1,0,1,0
4,68.0,80.0,0,1,0,1,0


Kolom Temperature dikelompokkan ke dalam dua kategori (warm dan Hot)

warm untuk nilai antara 0 - 80
Hot untuk nilai antara 80 - 100
Kolom ini dikonversi ke one-hot encoding, tetapi dengan drop_first=True, hanya kolom Hot yang disertakan (jika Temperature = warm, maka Hot = 0).
Kolom Humidity dikelompokkan ke dalam dua kategori (Dry dan Humid)

Dry untuk nilai antara 0 - 75
Humid untuk nilai antara 75 - 100
Dengan drop_first=True, hanya kolom Humid yang disertakan (Humid = 0 berarti Dry).
One-hot encoding diterapkan pada kolom Temperature dan Humidity

Kolom asli dihapus.
Kolom biner baru (Hot dan Humid) ditambahkan.

In [17]:
df['Temperature'] = pd.cut(df['Temperature'], bins=[0, 80, 100], labels=['warm', 'Hot'])
df['Humidity'] = pd.cut(df['Humidity'], bins=[0, 75, 100], labels=['Dry', 'Humid'])

one_hot_columns = pd.get_dummies(df[['Temperature', 'Humidity']], drop_first=True,dtype=int)
df = df.drop(['Temperature', 'Humidity'], axis=1)
df = pd.concat([df, one_hot_columns], axis=1)

df.head()

,Wind,Play,overcast,rainy,sunny,Temperature_Hot,Humidity_Humid
0,0,0,0,0,1,1,1
1,1,0,0,0,1,0,1
2,0,1,1,0,0,1,1
3,0,1,0,1,0,0,1
4,0,1,0,1,0,0,1


28 baris dalam dataset.
Semua kolom bertipe integer (int64) karena telah dikonversi ke numerik.
Tidak ada nilai kosong (non-null count = 28 untuk semua kolom).
Kolom Play adalah label (target), sementara kolom lainnya adalah fitur input untuk model Naïve Bayes.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Wind             28 non-null     int64
 1   Play             28 non-null     int64
 2   overcast         28 non-null     int64
 3   rainy            28 non-null     int64
 4   sunny            28 non-null     int64
 5   Temperature_Hot  28 non-null     int64
 6   Humidity_Humid   28 non-null     int64
dtypes: int64(7)
memory usage: 1.7 KB


Kode ini membagi dataset menjadi fitur (X) dan label (y), lalu melakukan pemisahan data untuk training dan testing dengan rasio 30% training dan 70% testing, tanpa pengacakan (shuffle=False).

In [25]:
X, y = df.drop(columns='Play'), df['Play']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

Menghitung jumlah total data → total_counts = attr.value_counts().sum().
Menghitung jumlah kemunculan tiap kelas dalam y_train → attr.value_counts().
Membagi tiap jumlah kemunculan dengan total data untuk mendapatkan probabilitasnya → Fraction(x, total_counts).
Menggunakan .limit_denominator() untuk membatasi bentuk pecahan agar lebih sederhana.

In [26]:
from fractions import Fraction

def cal_target_prob(attr):
  total_counts = attr.value_counts().sum()
  prob_series = attr.value_counts().apply(lambda x: Fraction(x, total_counts).limit_denominator())
  return prob_series

print(cal_target_prob(y_train))

Play
1    11/19
0     8/19
Name: count, dtype: object
